In [1]:
import sys
sys.path.append(r"C:\Users\sarwj\OneDrive - Cardiff University\Documents\GitHub\topologicpy\src") # change the path
from topologicpy.Vertex import Vertex
from topologicpy.Wire import Wire
from topologicpy.Cell import Cell
from topologicpy.CellComplex import CellComplex
from topologicpy.Cluster import Cluster
from topologicpy.Topology import Topology
from topologicpy.Dictionary import Dictionary
from topologicpy.Graph import Graph
from topologicpy.Plotly import Plotly
from topologicpy.BVH import BVH

In [5]:
cc = CellComplex.Prism(width=10, length=5, height=4, uSides=5, vSides=5, wSides=5)
exploded = Topology.Explode(cc, scale=2)
bounding_box = Topology.BoundingBox(exploded)
d = Topology.Dictionary(bounding_box)
min_x = Dictionary.ValueAtKey(d, "minx")
min_y = Dictionary.ValueAtKey(d, "miny")
min_z = Dictionary.ValueAtKey(d, "minz")
max_x = Dictionary.ValueAtKey(d, "maxx")
max_y = Dictionary.ValueAtKey(d, "maxy")
max_z = Dictionary.ValueAtKey(d, "maxz")
sv = Vertex.ByCoordinates(min_x, min_y, min_z)
ev = Vertex.ByCoordinates(max_x, max_y, max_z)
exploded_cells = Topology.Cells(exploded)
bvh = BVH.ByTopologies(exploded_cells)
graph = BVH.Graph(bvh)
sv = Graph.NearestVertex(graph, sv)
ev = Graph.NearestVertex(graph, ev)
shortest_path = Graph.ShortestPath(graph, sv, ev)
origins = []
steps = 30
for i in range(0,steps+1,1):
    origin = Wire.VertexByParameter(shortest_path, i/(steps))
    cells = [Cell.Prism(origin = origin, width=2, length=2, height=2)]
    
    query = BVH.QueryByTopologies(cells)
    clashing_cluster = Cluster.ByTopologies([Vertex.Origin()])
    clashing_objects = BVH.Clashes(bvh, query)
    if len(clashing_objects) > 0:
        clashing_cluster = Cluster.ByTopologies(clashing_objects)

    # Add BVH tree to the TopologicPy Graph
    # graph = add_bvh_to_graph(bvh_root, graph)
    data00 = Plotly.DataByTopology(Cluster.ByTopologies(cells), faceOpacity=0.2, faceColor="red", vertexSize=6)
    data01= Plotly.DataByGraph(graph, vertexSize=8, vertexColor="red", edgeWidth=4, edgeColor="blue")
    data02 = Plotly.DataByTopology(exploded, faceOpacity=0.1)
    data03 = Plotly.DataByTopology(clashing_cluster, faceOpacity=1, faceColor="yellow", edgeWidth=4)

    figure = Plotly.FigureByData(data00+data01+data02+data03, width=1024, height=960, backgroundColor='white')
    figure = Plotly.SetCamera(figure)
    status = Plotly.FigureExportToPNG(figure, "C:/Users/sarwj/Downloads/bvh"+str(i+1).zfill(3)+".png", overwrite=True)
    #Plotly.Show(figure)
